In [1]:
from transformers import AutoTokenizer, AutoModel, EsmForTokenClassification
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
import evaluate
import torch
from Bio import SeqIO
import pandas as pd
import numpy as np
from datasets import Dataset

In [2]:
tokenizer = AutoTokenizer.from_pretrained("facebook/esm2_t6_8M_UR50D")
model = EsmForTokenClassification.from_pretrained("facebook/esm2_t6_8M_UR50D", num_labels = 9)

Some weights of EsmForTokenClassification were not initialized from the model checkpoint at facebook/esm2_t6_8M_UR50D and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
train_path = 'train.tsv'
train_df = pd.read_csv(train_path, sep='\t')
train_df[['id','amino_acid','index']] = train_df['id'].str.split('_', expand=True)
train_df['index'] = train_df['index'].astype(int)

# Labels:
idx_to_labels = {i: value for i, value in enumerate(train_df['secondary_structure'].unique())}
labels_to_idx = {v: k for k, v in idx_to_labels.items()}
train_df['label'] = pd.factorize(train_df['secondary_structure'])[0]

# Ids:
ids = train_df['id'].unique()

In [4]:
seq_dict = SeqIO.to_dict(SeqIO.parse("sequences.fasta", "fasta"))

In [5]:
train_df['sequence'] = train_df['id'].apply(lambda x: str(seq_dict[x].seq))
train_df['length'] = train_df['sequence'].str.len()

In [6]:
train_df = train_df[train_df['index'] < train_df['length']]

In [7]:
df_filt = train_df.groupby('id')[['sequence']].agg({'sequence': 'first'}).reset_index()

In [8]:
df_shuffled = df_filt.sample(frac=1, random_state=1).reset_index(drop=True)

In [9]:
holdout = df_shuffled.iloc[: (len(df_filt) // 20)]
df_filt = df_shuffled.iloc[(len(df_filt) // 20) + 1:]

In [10]:
id_to_length = train_df.groupby('id')['length'].first().to_dict()
id_to_indices_labels = train_df.groupby('id')[['index', 'label']].agg(list).to_dict()

id_to_labels = {}
for id in id_to_length:
    length = id_to_length[id]
    indices = id_to_indices_labels['index'][id]
    labels = id_to_indices_labels['label'][id]
    model_labels = np.full(length, -100, dtype=np.int64)

    train_idx = np.array(indices) - 1  # Adjust for zero indexing
    train_labels = np.array(labels)

    model_labels[train_idx] = train_labels
    id_to_labels[id] = model_labels

In [11]:
df_filt['labels'] = df_filt['id'].map(id_to_labels)

/tmp/ipykernel_72508/3879309684.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filt['labels'] = df_filt['id'].map(id_to_labels)


In [12]:
def sliding_window_with_labels(sequence, labels, window_size, step_size):
    windows = []
    seq_len = len(sequence)

    for start in range(0, seq_len - window_size + 1, step_size):
        end = start + window_size
        windows.append((sequence[start:end], labels[start:end]))

    if (seq_len - window_size) % step_size != 0:
        start = max(seq_len - window_size, 0)
        end = seq_len
        windows.append((sequence[start:end], labels[start:end]))

    return windows

def create_sliding_windows(df, window_size, step_size):
    windows_data = []

    for _, row in df.iterrows():
        sequence = row['sequence']
        labels = row['labels']
        
        labels = np.array(labels)
        windows = sliding_window_with_labels(sequence, labels, window_size, step_size)
        
        for window_seq, window_labels in windows:
            windows_data.append({
                'id': row['id'],
                'sequence': window_seq,
                'labels': window_labels
            })
    
    windows_df = pd.DataFrame(windows_data)
    return windows_df


In [13]:
windows_8_4 = create_sliding_windows(df_filt, 8, 4)
windows_16_8 = create_sliding_windows(df_filt, 16, 8)
windows_32_16 = create_sliding_windows(df_filt, 32, 16)
windows_64_32 = create_sliding_windows(df_filt, 64, 32)
windows_128_64 = create_sliding_windows(df_filt, 128, 64)
windows_256_128 = create_sliding_windows(df_filt, 256, 128)

In [14]:
#windows_df_small = pd.concat([windows_8_4, windows_16_8], ignore_index=True)
windows_df_med = pd.concat([windows_32_16, windows_64_32], ignore_index=True)
windows_df_large = pd.concat([windows_64_32, windows_128_64], ignore_index=True)

In [15]:
#windows_ds_small = Dataset.from_pandas(windows_df_small).remove_columns('id')
windows_ds_med = Dataset.from_pandas(windows_df_med).remove_columns('id')
windows_ds_large = Dataset.from_pandas(windows_df_large).remove_columns('id')
ds_full = Dataset.from_pandas(df_filt).remove_columns('id')

In [16]:
def tokenize_and_label(example):
    tokenized_input = tokenizer(example['sequence'], add_special_tokens=False, return_tensors="pt")
    tokenized_input['input_ids'] = tokenized_input['input_ids'][0] 
    tokenized_input['attention_mask'] = tokenized_input['attention_mask'][0] 
    tokenized_input['labels'] = example['labels']
    return tokenized_input

In [17]:
#tokenized_ds_small = windows_ds_small.map(tokenize_and_label).remove_columns('sequence')
tokenized_ds_med = windows_ds_med.map(tokenize_and_label).remove_columns('sequence')
tokenized_ds_large = windows_ds_large.map(tokenize_and_label).remove_columns('sequence')
tokenized_ds_full = ds_full.map(tokenize_and_label).remove_columns('sequence')

Map:   0%|          | 0/134003 [00:00<?, ? examples/s]

Map:   0%|          | 0/64225 [00:00<?, ? examples/s]

Map:   0%|          | 0/5847 [00:00<?, ? examples/s]

In [18]:
#tokenized_ds_small.set_format("pt", columns=["input_ids", "attention_mask", "labels"], output_all_columns=True)
tokenized_ds_med.set_format("pt", columns=["input_ids", "attention_mask", "labels"], output_all_columns=True)
tokenized_ds_large.set_format("pt", columns=["input_ids", "attention_mask", "labels"], output_all_columns=True)
tokenized_ds_full.set_format("pt", columns=["input_ids", "attention_mask", "labels"], output_all_columns=True)

In [19]:
import random

In [20]:
def create_splits(ds, num_splits = 5):
    splits = []
    ds_shuffled = ds.shuffle(seed=len(ds))
    split_size = len(ds_shuffled) // num_splits
    
    for i in range(num_splits - 1):
        splits.append(ds_shuffled.select(range(i*split_size, (i+1)*split_size)).train_test_split(test_size=0.2, seed=123))
    
    splits.append(ds_shuffled.select(range((num_splits - 1) * split_size, len(ds_shuffled))).train_test_split(test_size=0.2, seed=123))
    return splits

In [21]:
#splits_small = create_splits(tokenized_ds_small)
splits_med = create_splits(tokenized_ds_med)
splits_large = create_splits(tokenized_ds_large)
splits_full = create_splits(tokenized_ds_full)

In [22]:
from itertools import chain

accuracy = evaluate.load("accuracy")
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    true_predictions = [
        [p for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    
    true_labels = [
        [l for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    flat_predictions = list(chain.from_iterable(true_predictions))
    flat_labels = list(chain.from_iterable(true_labels))
    
    results = accuracy.compute(predictions=flat_predictions, references=flat_labels)

    return {
        "accuracy": results["accuracy"],
    }

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [23]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [24]:
from transformers import AutoModelForTokenClassification, get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup, get_cosine_with_hard_restarts_schedule_with_warmup

In [25]:
def train_on_splits(splits, name, num_splits = 5, batch_size = 8, eval_batch_size = 8, num_epochs = 4):
    tokenizer = AutoTokenizer.from_pretrained("facebook/esm2_t6_8M_UR50D")
    model = EsmForTokenClassification.from_pretrained("facebook/esm2_t6_8M_UR50D", num_labels = 9)

    for i in range(num_splits):
        optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5, weight_decay=0.01, betas=(0.9, 0.999), eps=1e-08)
        num_training_steps = len(splits[i]["train"]) // batch_size * num_epochs
        num_warmup_steps = int(0.1 * num_training_steps)  # 10% warmup
        
        scheduler = get_linear_schedule_with_warmup(
            optimizer,
            num_warmup_steps=num_warmup_steps,
            num_training_steps=num_training_steps
        )
        
        training_args = TrainingArguments(
            output_dir = name + '_' + str(i),
            per_device_train_batch_size=batch_size,
            per_device_eval_batch_size=eval_batch_size,
            num_train_epochs=num_epochs,
            eval_strategy="epoch",
            save_strategy="epoch",
            logging_strategy="epoch",
            load_best_model_at_end=True,
        )
        
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=splits[i]["train"],
            eval_dataset=splits[i]["test"],
            processing_class=tokenizer,
            data_collator=data_collator,
            compute_metrics=compute_metrics,
            optimizers = (optimizer, scheduler),
        )
        
        trainer.can_return_loss = True
        trainer.train()
        
        model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
        model_to_save.save_pretrained(name + '_' + str(i))
        model = AutoModelForTokenClassification.from_pretrained(name + '_' + str(i))
    
        gc.collect()
        torch.cuda.empty_cache()


In [26]:
def train_on_splits_sequential(splits, name, model, num_splits = 5, batch_size = 8, eval_batch_size = 8, num_epochs = 4):
    tokenizer = AutoTokenizer.from_pretrained("facebook/esm2_t6_8M_UR50D")

    for i in range(num_splits):
        optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5, weight_decay=0.01, betas=(0.9, 0.999), eps=1e-08)
        num_training_steps = len(splits[i]["train"]) // batch_size * num_epochs
        num_warmup_steps = int(0.1 * num_training_steps)  # 10% warmup
        
        scheduler = get_linear_schedule_with_warmup(
            optimizer,
            num_warmup_steps=num_warmup_steps,
            num_training_steps=num_training_steps
        )
        
        training_args = TrainingArguments(
            output_dir = name + '_' + str(i),
            per_device_train_batch_size=batch_size,
            per_device_eval_batch_size=eval_batch_size,
            num_train_epochs=num_epochs,
            eval_strategy="epoch",
            save_strategy="epoch",
            logging_strategy="epoch",
            load_best_model_at_end=True,
        )
        
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=splits[i]["train"],
            eval_dataset=splits[i]["test"],
            processing_class=tokenizer,
            data_collator=data_collator,
            compute_metrics=compute_metrics,
            optimizers = (optimizer, scheduler),
        )
        
        trainer.can_return_loss = True
        trainer.train()
        
        model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
        model_to_save.save_pretrained(name + '_' + str(i))
        model = AutoModelForTokenClassification.from_pretrained(name + '_' + str(i))
    
        gc.collect()
        torch.cuda.empty_cache()

In [87]:
train_on_splits(splits_small, 'SMALL', batch_size = 64, eval_batch_size = 32)

Some weights of EsmForTokenClassification were not initialized from the model checkpoint at facebook/esm2_t6_8M_UR50D and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(

Epoch,Training Loss,Validation Loss,Accuracy
1,1.594500,1.432325,0.490741
2,1.419000,1.407922,0.496634
3,1.393100,1.392535,0.502865
4,1.379600,1.390555,0.503113


/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either 

Epoch,Training Loss,Validation Loss,Accuracy
1,1.403600,1.398084,0.500257
2,1.384900,1.389963,0.503873
3,1.369000,1.386982,0.505728
4,1.357000,1.386315,0.505816


/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either 

...

/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either 

Epoch,Training Loss,Validation Loss,Accuracy
1,1.375900,1.373788,0.506847
2,1.360900,1.371253,0.508469
3,1.345900,1.372390,0.507693
4,1.333600,1.374565,0.507525


/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either 

Epoch,Training Loss,Validation Loss,Accuracy
1,1.368900,1.375705,0.507543
2,1.354300,1.375005,0.507534
3,1.338500,1.374720,0.507863
4,1.326000,1.375407,0.507663


/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either 

In [95]:
gc.collect()
torch.cuda.empty_cache()
train_on_splits(splits_med, 'MED', batch_size = 32, eval_batch_size = 16)

Some weights of EsmForTokenClassification were not initialized from the model checkpoint at facebook/esm2_t6_8M_UR50D and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(

Epoch,Training Loss,Validation Loss,Accuracy
1,1.551900,1.317934,0.552529
2,1.278700,1.270476,0.561821
3,1.237200,1.251960,0.566645
4,1.217100,1.247063,0.568557


/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either 

Epoch,Training Loss,Validation Loss,Accuracy
1,1.249200,1.233718,0.571530
2,1.216000,1.235201,0.568143
3,1.190400,1.216748,0.575802
4,1.173300,1.217312,0.575493


/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either 

...

/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either 

Epoch,Training Loss,Validation Loss,Accuracy
1,1.191200,1.191289,0.583679
2,1.164500,1.185326,0.585035
3,1.139400,1.187752,0.585309
4,1.122500,1.188389,0.585154


/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either 

Epoch,Training Loss,Validation Loss,Accuracy
1,1.176200,1.172108,0.589473
2,1.151500,1.169810,0.590716
3,1.126800,1.170787,0.591212
4,1.109200,1.172383,0.591225


/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either 

In [99]:
gc.collect()
torch.cuda.empty_cache()
train_on_splits(splits_large, 'LARGE', batch_size = 32, eval_batch_size = 16)

Some weights of EsmForTokenClassification were not initialized from the model checkpoint at facebook/esm2_t6_8M_UR50D and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(

Epoch,Training Loss,Validation Loss,Accuracy
1,1.669000,1.349055,0.553325
2,1.292700,1.278377,0.569482
3,1.242800,1.256199,0.574542
4,1.221500,1.248993,0.575931


/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either 

Epoch,Training Loss,Validation Loss,Accuracy
1,1.234400,1.212017,0.584659
2,1.193600,1.200984,0.586592
3,1.168300,1.180763,0.592496
4,1.153600,1.178746,0.593019


/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either 

...

/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either 

Epoch,Training Loss,Validation Loss,Accuracy
1,1.151400,1.142372,0.602592
2,1.127000,1.134463,0.605879
3,1.106900,1.132644,0.607157
4,1.094000,1.132817,0.607540


/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either 

Epoch,Training Loss,Validation Loss,Accuracy
1,1.128500,1.123622,0.608449
2,1.105400,1.117731,0.610568
3,1.085500,1.118349,0.610418
4,1.072600,1.116277,0.610805


/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either 

In [48]:
gc.collect()
torch.cuda.empty_cache()
train_on_splits(splits_full, 'FULL', batch_size = 4, eval_batch_size = 4)

Some weights of EsmForTokenClassification were not initialized from the model checkpoint at facebook/esm2_t6_8M_UR50D and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(

Epoch,Training Loss,Validation Loss,Accuracy
1,1.766000,1.424772,0.535489
2,1.319400,1.302963,0.570147
3,1.237200,1.268079,0.579566
4,1.202600,1.259258,0.580801


/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either 

Epoch,Training Loss,Validation Loss,Accuracy
1,1.221100,1.210413,0.590954
2,1.163400,1.175169,0.599139
3,1.125300,1.163094,0.601722
4,1.104300,1.158343,0.603241


/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either 

...

/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either 

Epoch,Training Loss,Validation Loss,Accuracy
1,1.098100,1.058967,0.635158
2,1.062800,1.051987,0.635621
3,1.032900,1.049376,0.636566
4,1.015200,1.050520,0.636067


/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either 

Epoch,Training Loss,Validation Loss,Accuracy
1,1.087100,1.083987,0.622884
2,1.051800,1.081236,0.622811
3,1.021900,1.086322,0.623212
4,1.004600,1.083063,0.622957


/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either 

In [75]:
gc.collect()
torch.cuda.empty_cache()

In [27]:
gc.collect()
torch.cuda.empty_cache()
train_on_splits_sequential(splits_med, 'SMALL_MED',  model = AutoModelForTokenClassification.from_pretrained('SMALL_4'), 
                           batch_size = 128, eval_batch_size = 128)

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)


Epoch,Training Loss,Validation Loss,Accuracy
1,1.230700,1.215166,0.572857
2,1.207500,1.208033,0.576526
3,1.195100,1.205444,0.577311
4,1.187300,1.203585,0.577467


/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either 

Epoch,Training Loss,Validation Loss,Accuracy
1,1.201400,1.207799,0.575028
2,1.187000,1.204294,0.577039
3,1.175100,1.202103,0.577080
4,1.167700,1.202682,0.577753


/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either 

...

/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either 

Epoch,Training Loss,Validation Loss,Accuracy
1,1.189600,1.180373,0.585177
2,1.176300,1.176790,0.587213
3,1.164600,1.175749,0.587370
4,1.158100,1.175457,0.587504


/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either 

Epoch,Training Loss,Validation Loss,Accuracy
1,1.177300,1.169686,0.590403
2,1.163700,1.166936,0.590944
3,1.151200,1.166812,0.590815
4,1.143800,1.166649,0.591127


/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either 

In [36]:
gc.collect()
torch.cuda.empty_cache()
train_on_splits_sequential(splits_large, '/u/scratch/t/ttthach/P1/SMALL_MED_LARGE',  model = AutoModelForTokenClassification.from_pretrained('/u/scratch/t/ttthach/P1/SMALL_MED_4'), 
                           batch_size = 128, eval_batch_size = 128)

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)


Epoch,Training Loss,Validation Loss,Accuracy
1,1.116000,1.099093,0.615941
2,1.101800,1.096191,0.616676
3,1.093400,1.094045,0.617814
4,1.087100,1.094155,0.617564


/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either 

Epoch,Training Loss,Validation Loss,Accuracy
1,1.110300,1.105067,0.612663
2,1.099000,1.102254,0.614273
3,1.087900,1.100756,0.614536
4,1.082900,1.100690,0.614799


/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either 

...

/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either 

Epoch,Training Loss,Validation Loss,Accuracy
1,1.096600,1.106107,0.612682
2,1.083300,1.105044,0.613778
3,1.073600,1.104450,0.613992
4,1.067100,1.105006,0.613978


/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either 

Epoch,Training Loss,Validation Loss,Accuracy
1,1.093000,1.106647,0.610410
2,1.079700,1.102345,0.611182
3,1.069600,1.102968,0.611433
4,1.063900,1.102060,0.612008


/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either 

In [38]:
gc.collect()
torch.cuda.empty_cache()
train_on_splits_sequential(splits_full, '/u/scratch/t/ttthach/P1/SMALL_MED_LARGE_FULL',  model = AutoModelForTokenClassification.from_pretrained('/u/scratch/t/ttthach/P1/SMALL_MED_LARGE_4'), 
                           batch_size = 4, eval_batch_size = 4)

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)


Epoch,Training Loss,Validation Loss,Accuracy
1,1.014600,1.052108,0.627503
2,0.982200,1.051482,0.628961
3,0.952700,1.056971,0.628241
4,0.932600,1.060925,0.627263


/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either 

Epoch,Training Loss,Validation Loss,Accuracy
1,1.020000,1.039116,0.630042
2,0.990800,1.033415,0.632962
3,0.964800,1.034239,0.631595
4,0.945000,1.036229,0.631831


/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either 

...

/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either 

Epoch,Training Loss,Validation Loss,Accuracy
1,1.012900,0.979284,0.654520
2,0.983400,0.977719,0.653771
3,0.955100,0.980655,0.653432
4,0.937400,0.982571,0.653592


/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either 

Epoch,Training Loss,Validation Loss,Accuracy
1,1.016100,1.021339,0.640397
2,0.984500,1.020484,0.641127
3,0.953800,1.027319,0.640689
4,0.935300,1.029371,0.640342


/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either 

In [40]:
gc.collect()
torch.cuda.empty_cache()
train_on_splits_sequential(splits_large, '/u/scratch/t/ttthach/P1/MED_LARGE',  model = AutoModelForTokenClassification.from_pretrained('/u/scratch/t/ttthach/P1/MED_4'), 
                           batch_size = 128, eval_batch_size = 128)

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)


Epoch,Training Loss,Validation Loss,Accuracy
1,1.095400,1.079072,0.624380
2,1.080400,1.074320,0.626003
3,1.071100,1.072306,0.626499
4,1.064500,1.072463,0.626450


/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either 

Epoch,Training Loss,Validation Loss,Accuracy
1,1.088600,1.084376,0.622294
2,1.076600,1.082460,0.623203
3,1.065000,1.081820,0.623301
4,1.059900,1.081356,0.623870


/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either 

...

/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either 

Epoch,Training Loss,Validation Loss,Accuracy
1,1.077800,1.086046,0.621999
2,1.063400,1.084369,0.623368
3,1.053200,1.084196,0.623719
4,1.046200,1.084444,0.623622


/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either 

Epoch,Training Loss,Validation Loss,Accuracy
1,1.073000,1.086235,0.619143
2,1.058400,1.083289,0.619940
3,1.047800,1.084415,0.620070
4,1.041600,1.083847,0.619867


/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either 

In [49]:
gc.collect()
torch.cuda.empty_cache()
train_on_splits_sequential(splits_full, '/u/scratch/t/ttthach/P1/MED_LARGE_FULL',  model = AutoModelForTokenClassification.from_pretrained('/u/scratch/t/ttthach/P1/MED_LARGE_4'), 
                           batch_size = 4, eval_batch_size = 4)

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)


Epoch,Training Loss,Validation Loss,Accuracy
1,0.990300,1.029323,0.637505
2,0.957600,1.029736,0.636596
3,0.928000,1.035997,0.637111
4,0.907300,1.038994,0.636390


/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either 

Epoch,Training Loss,Validation Loss,Accuracy
1,0.997800,1.014990,0.640540
2,0.968900,1.013240,0.640827
3,0.942900,1.013454,0.641941
4,0.923400,1.015615,0.641789


/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either 

...

/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either 

Epoch,Training Loss,Validation Loss,Accuracy
1,0.990600,0.957836,0.662952
2,0.960300,0.958540,0.662061
3,0.931400,0.963865,0.659636
4,0.913400,0.965974,0.659565


/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either 

Epoch,Training Loss,Validation Loss,Accuracy
1,0.993400,1.001542,0.647895
2,0.961500,1.001965,0.646946
3,0.930800,1.010912,0.647493
4,0.911900,1.013103,0.646399


/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either 

In [53]:
gc.collect()
torch.cuda.empty_cache()
train_on_splits_sequential(splits_full, '/u/scratch/t/ttthach/P1/LARGE_FULL',  model = AutoModelForTokenClassification.from_pretrained('/u/scratch/t/ttthach/P1/LARGE_4'), 
                           batch_size = 4, eval_batch_size = 4)

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)


Epoch,Training Loss,Validation Loss,Accuracy
1,1.020000,1.055909,0.630745
2,0.987300,1.055037,0.630059
3,0.957900,1.060589,0.630196
4,0.937700,1.062977,0.630145


/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either 

Epoch,Training Loss,Validation Loss,Accuracy
1,1.022100,1.041839,0.633013
2,0.995100,1.037011,0.633232
3,0.968700,1.037691,0.633131
4,0.950500,1.039464,0.633232


/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either 

...

/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either 

Epoch,Training Loss,Validation Loss,Accuracy
1,1.013700,0.981804,0.656249
2,0.984200,0.980771,0.656053
3,0.956900,0.983512,0.655108
4,0.939800,0.985319,0.655268


/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either 

Epoch,Training Loss,Validation Loss,Accuracy
1,1.017200,1.017091,0.643316
2,0.984500,1.016937,0.644392
3,0.954800,1.023861,0.643754
4,0.936700,1.023006,0.643170


/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either 

In [55]:
gc.collect()
torch.cuda.empty_cache()
train_on_splits_sequential(splits_full, '/u/scratch/t/ttthach/P1/SMALL_FULL',  model = AutoModelForTokenClassification.from_pretrained('/u/scratch/t/ttthach/P1/SMALL_4'), 
                           batch_size = 4, eval_batch_size = 4)

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)


Epoch,Training Loss,Validation Loss,Accuracy
1,1.167300,1.165140,0.594235
2,1.102900,1.149861,0.597684
3,1.070200,1.145555,0.599520
4,1.050900,1.144315,0.600412


/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either 

Epoch,Training Loss,Validation Loss,Accuracy
1,1.110600,1.116447,0.608321
2,1.079200,1.108173,0.611156
3,1.053400,1.105051,0.612203
4,1.035200,1.103676,0.612051


/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either 

...

/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either 

Epoch,Training Loss,Validation Loss,Accuracy
1,1.070700,1.034901,0.638563
2,1.041600,1.030194,0.639330
3,1.014900,1.030031,0.639526
4,0.998500,1.030325,0.639597


/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either 

Epoch,Training Loss,Validation Loss,Accuracy
1,1.070700,1.070051,0.625091
2,1.038300,1.068637,0.626003
3,1.009600,1.076935,0.624307
4,0.993500,1.072472,0.625164


/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either 

In [61]:
gc.collect()
torch.cuda.empty_cache()
train_on_splits_sequential(splits_full, '/u/scratch/t/ttthach/P1/MED_FULL',  model = AutoModelForTokenClassification.from_pretrained('/u/scratch/t/ttthach/P1/MED_4'), 
                           batch_size = 4, eval_batch_size = 4)

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)


Epoch,Training Loss,Validation Loss,Accuracy
1,1.030100,1.064976,0.625650
2,0.995000,1.064351,0.625066
3,0.966400,1.067723,0.626113
4,0.947000,1.070214,0.625084


/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either 

Epoch,Training Loss,Validation Loss,Accuracy
1,1.029400,1.044552,0.631814
2,1.001200,1.040045,0.633755
3,0.976100,1.040055,0.633401
4,0.957700,1.041577,0.634076


/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either 

...

/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either 

Epoch,Training Loss,Validation Loss,Accuracy
1,1.015600,0.983757,0.654199
2,0.986900,0.983384,0.654270
3,0.959800,0.986960,0.653378
4,0.942900,0.988920,0.652612


/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either 

Epoch,Training Loss,Validation Loss,Accuracy
1,1.018600,1.023105,0.641254
2,0.987200,1.023357,0.640817
3,0.957600,1.030679,0.640689
4,0.940100,1.031882,0.639357


/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either 

In [31]:
gc.collect()
torch.cuda.empty_cache()

holdout['labels'] = holdout['id'].map(id_to_labels)
holdout_ds = Dataset.from_pandas(holdout).remove_columns('id')
tokenized_ds_holdout = holdout_ds.map(tokenize_and_label).remove_columns('sequence')
tokenized_ds_holdout.set_format("pt", columns=["input_ids", "attention_mask", "labels"], output_all_columns=True)

/tmp/ipykernel_72508/3696022308.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  holdout['labels'] = holdout['id'].map(id_to_labels)


Map:   0%|          | 0/307 [00:00<?, ? examples/s]

In [29]:
def create_predictions(ds, model, tokenizer, data_collator, compute_metrics, dir_name):
    test_args = TrainingArguments(
        output_dir = dir_name,
        do_train = False,
        do_predict = True,
        per_device_eval_batch_size = 4,
        dataloader_drop_last = False
    )
    
    # init trainer
    trainer = Trainer(
                  model = model,
                  args = test_args,
                  processing_class=tokenizer,
                  data_collator=data_collator,
                  compute_metrics = compute_metrics)
    
    results = trainer.predict(ds)
    return results

In [30]:
accuracy = evaluate.load("accuracy")
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    true_predictions = [
        [p for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    
    true_labels = [
        [l for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    flat_predictions = list(chain.from_iterable(true_predictions))
    flat_labels = list(chain.from_iterable(true_labels))
    
    results = accuracy.compute(predictions=flat_predictions, references=flat_labels)

    return {
        "accuracy": results["accuracy"],
    }

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)
tokenizer = AutoTokenizer.from_pretrained("facebook/esm2_t6_8M_UR50D")

In [34]:
small_results = create_predictions(tokenized_ds_holdout, AutoModelForTokenClassification.from_pretrained('/u/scratch/t/ttthach/P1/SMALL_4'), 
                                   tokenizer=tokenizer, data_collator=data_collator, compute_metrics=compute_metrics, 
                                   dir_name='/u/scratch/t/ttthach/P1/SMALL_VAL')
small_results.metrics

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)


{'test_loss': 1.2701047658920288,
 'test_model_preparation_time': 0.003,
 'test_accuracy': 0.5677826297913634,
 'test_runtime': 3.2317,
 'test_samples_per_second': 94.996,
 'test_steps_per_second': 23.826}

In [35]:
med_results = create_predictions(tokenized_ds_holdout, AutoModelForTokenClassification.from_pretrained('/u/scratch/t/ttthach/P1/MED_4'), 
                                   tokenizer=tokenizer, data_collator=data_collator, compute_metrics=compute_metrics, 
                                   dir_name='/u/scratch/t/ttthach/P1/MED_VAL')
med_results.metrics

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)


{'test_loss': 1.0722001791000366,
 'test_model_preparation_time': 0.0024,
 'test_accuracy': 0.6194357801344701,
 'test_runtime': 3.1858,
 'test_samples_per_second': 96.365,
 'test_steps_per_second': 24.17}

In [55]:
large_results = create_predictions(tokenized_ds_holdout, AutoModelForTokenClassification.from_pretrained('LARGE_4'), 
                                   tokenizer=tokenizer, data_collator=data_collator, compute_metrics=compute_metrics, 
                                   dir_name='LARGE_VAL')
large_results.metrics

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)


{'test_loss': 1.0640357732772827,
 'test_model_preparation_time': 0.0032,
 'test_accuracy': 0.625133430373605,
 'test_runtime': 2.03,
 'test_samples_per_second': 151.228,
 'test_steps_per_second': 37.93}

In [56]:
full_results = create_predictions(tokenized_ds_holdout, AutoModelForTokenClassification.from_pretrained('FULL_4'), 
                                   tokenizer=tokenizer, data_collator=data_collator, compute_metrics=compute_metrics, 
                                   dir_name='FULL_VAL')
full_results.metrics

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)


{'test_loss': 1.0721930265426636,
 'test_model_preparation_time': 0.0031,
 'test_accuracy': 0.6230401330838012,
 'test_runtime': 2.0259,
 'test_samples_per_second': 151.537,
 'test_steps_per_second': 38.008}

In [33]:
sm_results = create_predictions(tokenized_ds_holdout, AutoModelForTokenClassification.from_pretrained('/u/scratch/t/ttthach/P1/SMALL_MED_4'), 
                                   tokenizer=tokenizer, data_collator=data_collator, compute_metrics=compute_metrics, 
                                   dir_name='/u/scratch/t/ttthach/P1/SMALL_MED_VAL')
sm_results.metrics

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)


{'test_loss': 1.1020615100860596,
 'test_model_preparation_time': 0.0029,
 'test_accuracy': 0.6077909475289388,
 'test_runtime': 3.3362,
 'test_samples_per_second': 92.022,
 'test_steps_per_second': 23.08}

In [37]:
sml_results = create_predictions(tokenized_ds_holdout, AutoModelForTokenClassification.from_pretrained('/u/scratch/t/ttthach/P1/SMALL_MED_LARGE_4'), 
                                   tokenizer=tokenizer, data_collator=data_collator, compute_metrics=compute_metrics, 
                                   dir_name='/u/scratch/t/ttthach/P1/SMALL_MED_LARGE_VAL')
sml_results.metrics

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)


{'test_loss': 1.0752309560775757,
 'test_model_preparation_time': 0.0032,
 'test_accuracy': 0.6175781520759687,
 'test_runtime': 3.3017,
 'test_samples_per_second': 92.983,
 'test_steps_per_second': 23.322}

In [39]:
smlf_results = create_predictions(tokenized_ds_holdout, AutoModelForTokenClassification.from_pretrained('/u/scratch/t/ttthach/P1/SMALL_MED_LARGE_FULL_4'), 
                                   tokenizer=tokenizer, data_collator=data_collator, compute_metrics=compute_metrics, 
                                   dir_name='/u/scratch/t/ttthach/P1/SMALL_MED_LARGE_FULL_VAL')
smlf_results.metrics

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)


{'test_loss': 1.052809476852417,
 'test_model_preparation_time': 0.0032,
 'test_accuracy': 0.6277396548138906,
 'test_runtime': 3.2971,
 'test_samples_per_second': 93.112,
 'test_steps_per_second': 23.354}

In [41]:
ml_results = create_predictions(tokenized_ds_holdout, AutoModelForTokenClassification.from_pretrained('/u/scratch/t/ttthach/P1/MED_LARGE_4'), 
                                   tokenizer=tokenizer, data_collator=data_collator, compute_metrics=compute_metrics, 
                                   dir_name='/u/scratch/t/ttthach/P1/MED_LARGE_VAL')
ml_results.metrics

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)


{'test_loss': 1.0496424436569214,
 'test_model_preparation_time': 0.0034,
 'test_accuracy': 0.6273237679351217,
 'test_runtime': 3.291,
 'test_samples_per_second': 93.285,
 'test_steps_per_second': 23.397}

In [52]:
mlf_results = create_predictions(tokenized_ds_holdout, AutoModelForTokenClassification.from_pretrained('/u/scratch/t/ttthach/P1/MED_LARGE_FULL_4'), 
                                   tokenizer=tokenizer, data_collator=data_collator, compute_metrics=compute_metrics, 
                                   dir_name='/u/scratch/t/ttthach/P1/MED_LARGE_FULL_VAL')
mlf_results.metrics

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)


{'test_loss': 1.0312223434448242,
 'test_model_preparation_time': 0.0026,
 'test_accuracy': 0.6359187634296805,
 'test_runtime': 3.2167,
 'test_samples_per_second': 95.441,
 'test_steps_per_second': 23.938}

In [54]:
lf_results = create_predictions(tokenized_ds_holdout, AutoModelForTokenClassification.from_pretrained('/u/scratch/t/ttthach/P1/LARGE_FULL_4'), 
                                   tokenizer=tokenizer, data_collator=data_collator, compute_metrics=compute_metrics, 
                                   dir_name='/u/scratch/t/ttthach/P1/LARGE_FULL_VAL')
lf_results.metrics

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)


{'test_loss': 1.0366559028625488,
 'test_model_preparation_time': 0.0045,
 'test_accuracy': 0.6333541276772717,
 'test_runtime': 3.2933,
 'test_samples_per_second': 93.22,
 'test_steps_per_second': 23.381}

In [56]:
sf_results = create_predictions(tokenized_ds_holdout, AutoModelForTokenClassification.from_pretrained('/u/scratch/t/ttthach/P1/SMALL_FULL_4'), 
                                   tokenizer=tokenizer, data_collator=data_collator, compute_metrics=compute_metrics, 
                                   dir_name='/u/scratch/t/ttthach/P1/SMALL_FULL_VAL')
sf_results.metrics

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)


{'test_loss': 1.0622258186340332,
 'test_model_preparation_time': 0.0033,
 'test_accuracy': 0.6229985443959243,
 'test_runtime': 3.2846,
 'test_samples_per_second': 93.468,
 'test_steps_per_second': 23.443}

In [62]:
mf_results = create_predictions(tokenized_ds_holdout, AutoModelForTokenClassification.from_pretrained('/u/scratch/t/ttthach/P1/MED_FULL_4'), 
                                   tokenizer=tokenizer, data_collator=data_collator, compute_metrics=compute_metrics, 
                                   dir_name='/u/scratch/t/ttthach/P1/MED_FULL_VAL')
mf_results.metrics

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)


{'test_loss': 1.0335673093795776,
 'test_model_preparation_time': 0.003,
 'test_accuracy': 0.6340195466833022,
 'test_runtime': 3.2641,
 'test_samples_per_second': 94.052,
 'test_steps_per_second': 23.59}

In [62]:
avg_result = (0.33*med_results.predictions + 0.33*full_results.predictions + 0.34*large_results.predictions)

In [63]:
predictions = avg_result.argmax(axis=2)

In [64]:
labels = holdout['labels']

In [65]:
    true_predictions = [
        [p for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    
    true_labels = [
        [l for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]


In [66]:
    flat_predictions = list(chain.from_iterable(true_predictions))
    flat_labels = list(chain.from_iterable(true_labels))
    
    results = accuracy.compute(predictions=flat_predictions, references=flat_labels)
results

{'accuracy': 0.6286407430512234}

In [43]:
test_path = 'test.tsv'
test_df = pd.read_csv(test_path, sep='\t')
test_df[['id','amino_acid','index']] = test_df['id'].str.split('_', expand=True)
test_df['index'] = test_df['index'].astype(int)
test_df['sequence'] = test_df['id'].apply(lambda x: str(seq_dict[x].seq))
test_filt = test_df.groupby('id')[['sequence']].agg({'sequence': 'first'}).reset_index()

test_ds_filt = Dataset.from_pandas(test_filt).remove_columns('id')

In [44]:
def tokenize_and_convert(example):
    tokenized_input = tokenizer(example['sequence'], add_special_tokens=False, return_tensors="pt")
    tokenized_input['input_ids'] = tokenized_input['input_ids'][0] 
    tokenized_input['attention_mask'] = tokenized_input['attention_mask'][0] 
    return tokenized_input

In [45]:
tokenized_test_ds = test_ds_filt.map(tokenize_and_convert).remove_columns('sequence')
tokenized_test_ds.set_format("pt", columns=["input_ids", "attention_mask"], output_all_columns=True)

Map:   0%|          | 0/1552 [00:00<?, ? examples/s]

In [28]:
accuracy = evaluate.load("accuracy")
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    true_predictions = [
        [p for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    
    true_labels = [
        [l for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    flat_predictions = list(chain.from_iterable(true_predictions))
    flat_labels = list(chain.from_iterable(true_labels))
    
    results = accuracy.compute(predictions=flat_predictions, references=flat_labels)

    return {
        "accuracy": results["accuracy"],
    }

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)
tokenizer = AutoTokenizer.from_pretrained("facebook/esm2_t6_8M_UR50D")

In [31]:
small_test = create_predictions(tokenized_test_ds, AutoModelForTokenClassification.from_pretrained('SMALL_4'), 
                                   tokenizer=tokenizer, data_collator=data_collator, compute_metrics=compute_metrics, 
                                   dir_name='SMALL_TEST')
#y_pred = np.argmax(small_test.predictions, axis=2)

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)


In [32]:
med_test = create_predictions(tokenized_test_ds, AutoModelForTokenClassification.from_pretrained('MED_4'), 
                                   tokenizer=tokenizer, data_collator=data_collator, compute_metrics=compute_metrics, 
                                   dir_name='MED_TEST')
#y_pred = np.argmax(med_test.predictions, axis=2)

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)


In [64]:
large_test = create_predictions(tokenized_test_ds, AutoModelForTokenClassification.from_pretrained('/u/scratch/t/ttthach/P1/LARGE_4'), 
                                   tokenizer=tokenizer, data_collator=data_collator, compute_metrics=compute_metrics, 
                                   dir_name='/u/scratch/t/ttthach/P1/LARGE_TEST')
#y_pred = np.argmax(large_test.predictions, axis=2)

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)


In [65]:
full_test = create_predictions(tokenized_test_ds, AutoModelForTokenClassification.from_pretrained('/u/scratch/t/ttthach/P1/FULL_4'), 
                                   tokenizer=tokenizer, data_collator=data_collator, compute_metrics=compute_metrics, 
                                   dir_name='/u/scratch/t/ttthach/P1/FULL_TEST')
#y_pred = np.argmax(full_test.predictions, axis=2)

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)


In [63]:
smlf_test = create_predictions(tokenized_test_ds, AutoModelForTokenClassification.from_pretrained('/u/scratch/t/ttthach/P1/SMALL_MED_LARGE_FULL_4'), 
                                   tokenizer=tokenizer, data_collator=data_collator, compute_metrics=compute_metrics, 
                                   dir_name='/u/scratch/t/ttthach/P1/SMALL_MED_LARGE_FULL_TEST')

#y_pred = np.argmax(smlf_test.predictions, axis=2)

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)


In [66]:
ml_test = create_predictions(tokenized_test_ds, AutoModelForTokenClassification.from_pretrained('/u/scratch/t/ttthach/P1/MED_LARGE_4'), 
                                   tokenizer=tokenizer, data_collator=data_collator, compute_metrics=compute_metrics, 
                                   dir_name='/u/scratch/t/ttthach/P1/MED_LARGE_TEST')

#y_pred = np.argmax(mlf_test.predictions, axis=2)

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)


In [67]:
mlf_test = create_predictions(tokenized_test_ds, AutoModelForTokenClassification.from_pretrained('/u/scratch/t/ttthach/P1/MED_LARGE_FULL_4'), 
                                   tokenizer=tokenizer, data_collator=data_collator, compute_metrics=compute_metrics, 
                                   dir_name='/u/scratch/t/ttthach/P1/MED_LARGE_FULL_TEST')

#y_pred = np.argmax(mlf_test.predictions, axis=2)

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)


In [68]:
lf_test = create_predictions(tokenized_test_ds, AutoModelForTokenClassification.from_pretrained('/u/scratch/t/ttthach/P1/LARGE_FULL_4'), 
                                   tokenizer=tokenizer, data_collator=data_collator, compute_metrics=compute_metrics, 
                                   dir_name='/u/scratch/t/ttthach/P1/LARGE_FULL_TEST')


Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)


In [69]:
sf_test = create_predictions(tokenized_test_ds, AutoModelForTokenClassification.from_pretrained('/u/scratch/t/ttthach/P1/SMALL_FULL_4'), 
                                   tokenizer=tokenizer, data_collator=data_collator, compute_metrics=compute_metrics, 
                                   dir_name='/u/scratch/t/ttthach/P1/SMALL_FULL_TEST')


Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)


In [73]:
mf_test = create_predictions(tokenized_test_ds, AutoModelForTokenClassification.from_pretrained('/u/scratch/t/ttthach/P1/MED_FULL_4'), 
                                   tokenizer=tokenizer, data_collator=data_collator, compute_metrics=compute_metrics, 
                                   dir_name='/u/scratch/t/ttthach/P1/MED_FULL_TEST')

#y_pred = np.argmax(mf_test.predictions, axis=2)

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/u/home/t/ttthach/.local/lib/python3.10/site-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)


In [74]:
pred = (large_test.predictions + full_test.predictions + smlf_test.predictions + ml_test.predictions + mlf_test.predictions + lf_test.predictions + sf_test.predictions + mf_test.predictions) / 8

In [75]:
y_pred = np.argmax(pred, axis=2)

In [76]:
pred_df = pd.DataFrame(y_pred)

In [47]:
#test_prob = test_filt.copy() #.drop(columns=['predictions'])
#test_prob['prob'] = prob.apply(lambda x: x.tolist(), axis=1)
#prob_merge = pd.merge(test_df, test_prob, on=['id','sequence'], how='left')
#prob_merge['probability'] = prob_merge.apply(lambda x: x['prob'][x['index'] - 1], axis=1)
#prob_merge = pd.merge(test_df, test_prob, on=['id','sequence'], how='left')
#prob_merge['prob_list'] = prob_merge.apply(lambda x: x['prob'][9*(x['index'] - 1):9*(x['index'])], axis=1)
#test_prob_submit = pd.read_csv(test_path, sep='\t')
#test_prob_submit['prob'] = prob_merge['prob_list']
#test_prob_submit.to_csv('/u/scratch/t/ttthach/P1/test_prob.csv', sep='\t', index=False)
#prob_df = test_prob_submit['prob'].apply(pd.Series)

In [77]:
test_filt['predictions'] = pred_df.apply(lambda x: x.tolist(), axis=1)

test_df_merged = pd.merge(test_df, test_filt, on=['id','sequence'], how='left')

test_df_merged['class_number'] = test_df_merged.apply(lambda x: x['predictions'][x['index'] - 1], axis=1)

test_df_merged['secondary_structure'] = test_df_merged['class_number'].apply(lambda x: idx_to_labels[x])

test_df_submit = pd.read_csv(test_path, sep='\t')

test_df_submit['secondary_structure'] = test_df_merged['secondary_structure']

test_df_submit.to_csv('predictions.csv', sep='\t', index=False)

test_df_submit.head(50)

,id,secondary_structure
0,3JRN_LYS_8,.
1,3JRN_TYR_9,E
2,3JRN_ASP_10,E
3,3JRN_VAL_11,E
4,3JRN_PHE_12,E
5,3JRN_LEU_13,E
6,3JRN_SER_14,E
7,3JRN_PHE_15,E
8,3JRN_ARG_16,.
9,3JRN_GLY_17,.
